In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler



In [2]:
#loading the data 
df = pd.read_csv("milknew.csv")
df.head()

,pH,Temprature,Taste,Odor,Fat,Turbidity,Colour,Grade
0,6.6,35,1,0,1,0,254,high
1,6.6,36,0,1,0,1,253,high
2,8.5,70,1,1,1,1,246,low
3,9.5,34,1,1,0,1,255,low
4,6.6,37,0,0,0,0,255,medium


In [3]:
#loading the targets and the input features
# converting them to numpy arrays
y = df['Grade'].values
X = df.iloc[:,0:7].values

print(f"Type of X and Type of y:{type(X), type(y)}")

Type of X and Type of y:(<class 'numpy.ndarray'>, <class 'numpy.ndarray'>)


In [4]:
#splitting the data set into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

print(f"Shape of X_train and X_test:{X_train.shape, X_test.shape}")

print(f"Shape of y_train and y_test:{y_train.shape, y_test.shape}")

Shape of X_train and X_test:((847, 7), (212, 7))
Shape of y_train and y_test:((847,), (212,))


In [5]:
print(f"First row of X_train and X_test:{X_train[0], X_test[0]}")

print(f"First five elements of y_train and y_test:{y_train[0:5], y_test[0:5]}")

First row of X_train and X_test:(array([  7.4,  65. ,   0. ,   0. ,   0. ,   0. , 255. ]), array([  6.7,  38. ,   1. ,   0. ,   1. ,   0. , 255. ]))
First five elements of y_train and y_test:(array(['low', 'medium', 'high', 'medium', 'high'], dtype=object), array(['high', 'low', 'medium', 'medium', 'medium'], dtype=object))


In [6]:
#Label encoding y_test  and y_train

y_train = LabelEncoder().fit_transform(y_train) 
y_test = LabelEncoder().fit_transform(y_test)

print(f"Label encoded outputs of y_train and y_test:{y_train[0:5], y_test[0:5]}")

Label encoded outputs of y_train and y_test:(array([1, 2, 0, 2, 0]), array([0, 1, 2, 2, 2]))


In [7]:
#Scaling the input features of X_train and X_test
X_train = StandardScaler().fit_transform(X_train)
X_test = StandardScaler().fit_transform(X_test)

print(f"First row of X_train and X_test after feature scaling: {X_train[0], X_test[0]}")

First row of X_train and X_test after feature scaling: (array([ 0.55707032,  2.04010539, -1.08507045, -0.8660254 , -1.41546674,
       -0.96404614,  0.7261271 ]), array([-0.00426705, -0.65946559,  0.86721746, -0.90109146,  0.6723502 ,
       -1.05830052,  0.76562929]))


In [8]:
# creating a neural network model to determine the quality of the milk
#Linear activation outputs applied to softmax is more numerically stable
#compared to softmax activation
model = Sequential(
    [
        Dense(30,activation="relu"),
        Dense(15,activation="relu"),
        Dense(7,activation="relu"),
        Dense(3,activation="linear")
    ]
)

In [9]:
#compiling the model with the appropriate loss function and fitting the training data
model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = tf.keras.optimizers.Adam(0.002),
)

model.fit(
    X_train, y_train,
    epochs = 50
)

Epoch 1/50
27/27 [==============================] - 1s 3ms/step - loss: 0.9410
Epoch 2/50
27/27 [==============================] - 0s 3ms/step - loss: 0.6809
Epoch 3/50
27/27 [==============================] - 0s 2ms/step - loss: 0.5193
Epoch 4/50
27/27 [==============================] - 0s 3ms/step - loss: 0.4142
Epoch 5/50
27/27 [==============================] - 0s 2ms/step - loss: 0.3287
Epoch 6/50
27/27 [==============================] - 0s 2ms/step - loss: 0.2638
Epoch 7/50
27/27 [==============================] - 0s 2ms/step - loss: 0.2048
Epoch 8/50
27/27 [==============================] - 0s 2ms/step - loss: 0.1610
Epoch 9/50
27/27 [==============================] - 0s 2ms/step - loss: 0.1352
Epoch 10/50
27/27 [==============================] - 0s 2ms/step - loss: 0.1215
Epoch 11/50
27/27 [==============================] - 0s 2ms/step - loss: 0.1141
Epoch 12/50
27/27 [==============================] - 0s 2ms/step - loss: 0.1071
Epoch 13/50
27/27 [==============================

In [10]:
#here we obtain the predictions of the network

prediction = tf.nn.softmax(model.predict(X_test)).numpy()
c=0
for i in range(len(prediction)):
    if(np.argmax(prediction[i])!=y_test[i]):
        #print(f" target and prediction: {np.argmax(prediction[i]), y_test[i]}")
        c = c + 1
print(c/len(y_test))


7/7 [==============================] - 0s 1ms/step
0.02358490566037736


In [11]:
pred = tf.nn.softmax(model.predict(X_train)).numpy()

c=0

for i in range(len(pred)):
    if(np.argmax(pred[i])!=y_train[i]):
        c += 1
print(c/len(y_train))

27/27 [==============================] - 0s 2ms/step
0.0
